In [55]:
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import optuna
import os
from dateutil.relativedelta import relativedelta

## Load Data

In [116]:
data_path = '../data/Predicting Coupon Redemption/train'
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
demo_df = pd.read_csv(os.path.join(data_path, 'customer_demographics.csv'))
cmpn_df = pd.read_csv(os.path.join(data_path, 'campaign_data.csv'))
cp_it_df = pd.read_csv(os.path.join(data_path, 'coupon_item_mapping.csv'))
txn_df = pd.read_csv(os.path.join(data_path, 'customer_transaction_data.csv'))
item_df = pd.read_csv(os.path.join(data_path, 'item_data.csv'))

In [117]:
label_df = pd.read_parquet(os.path.join(data_path, 'label.parquet'))
label_df.head()

,customer_id,txn_month,Alcohol,Bakery,DairyJuicesSnacks,FlowersPlants,Fuel,Garden,Grocery,Meat,...,NaturalProducts,PackagedMeat,Pharmaceutical,PreparedFood,Restauarant,Salads,Seafood,SkinHairCare,Travel,Vegetablescut
0,1,2012-02,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2012-03,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2012-04,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2012-05,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2012-06,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
index_cols = ['customer_id', 'txn_month']

In [119]:
index_df = label_df.loc[label_df['txn_month']>'2012-03', index_cols]

## Demographics

In [120]:
join_df = index_df.merge(demo_df, on='customer_id', how='left')
join_df.head()

,customer_id,txn_month,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,2012-04,70+,Married,0.0,2,NaN,4.0
1,1,2012-05,70+,Married,0.0,2,NaN,4.0
2,1,2012-06,70+,Married,0.0,2,NaN,4.0
3,1,2012-07,70+,Married,0.0,2,NaN,4.0
4,1,2012-08,70+,Married,0.0,2,NaN,4.0


In [121]:
feat_cols = ['age_range', 'marital_status', 'rented', 'family_size', 'no_of_children', 'income_bracket']
cat_cols = ['age_range', 'marital_status','family_size', 'no_of_children']
numer_cols = ['rented', 'income_bracket']

## Transaction History Features

In [122]:
txn_df = txn_df.merge(item_df[['item_id', 'category']], on='item_id', how='left')
txn_df.shape

(1324566, 8)

In [123]:
# cleaning 
print('duplicated rows: ', txn_df.duplicated().sum())
print('drop duplicate rows')
txn_df.drop_duplicates(inplace=True)
print('duplicated rows: ', txn_df.duplicated().sum())

print('cleaning category name by removing space and symbols')
categories = txn_df['category'].unique()
new_categories = [re.sub('\s+', "", re.sub("[^A-Za-z0-9_ ]", "", cat_nm)) for cat_nm in categories]
cat_nm_map = dict(zip(categories, new_categories))
txn_df['category'] = txn_df['category'].map(cat_nm_map)
txn_df['category'].value_counts(dropna=False)

txn_df['txn_month'] = txn_df['date'].str[:7]

duplicated rows:  2916
drop duplicate rows
duplicated rows:  0
cleaning category name by removing space and symbols


In [166]:
def get_previous_month(current, p=1):
    curr_month = dt.datetime.strptime(current, "%Y-%m")
    previous_month = curr_month-relativedelta(months=p)
    return previous_month.strftime("%Y-%m")

def get_feature(index_df, feature_df, p=1):
    index_df = index_df.copy()
    index_df['join_month'] = index_df['txn_month'].apply(get_previous_month, p)
    tmp_feature = index_df.merge(feature_df, left_on=['customer_id', 'join_month'], 
                                 right_on=['customer_id', 'txn_month'], how='left')
    tmp_feature.drop('join_month', axis=1, inplace=True)    
    print(f'delete column {tmp_feature.columns[2]}')
    tmp_feature.pop(tmp_feature.columns[2])
    cols = tmp_feature.columns[2:]
    new_cols = [f'{col}_P{p}M' for col in cols ]
    tmp_feature.columns = index_cols + new_cols
    return tmp_feature

def get_purchase_freq(df):
    for c in classes:
        cols = [f'purchase_{c}_cnt_P{i}M' for i in range(1,4)]
        col_nm = f'{c}_freq'
        df[col_nm] = (df[cols]>0).sum(axis=1)
    return df

In [144]:
agg_txn = pd.pivot_table(data=txn_df, 
                         values=['quantity','selling_price'], 
                         index=['customer_id', 'txn_month'],
                         columns='category',
                         aggfunc=np.sum)
agg_txn.columns = agg_txn.columns.to_series().str.join('_')
agg_txn = agg_txn.reset_index()
agg_txn.head()

,customer_id,txn_month,quantity_Alcohol,quantity_Bakery,quantity_DairyJuicesSnacks,quantity_FlowersPlants,quantity_Fuel,quantity_Garden,quantity_Grocery,quantity_Meat,...,selling_price_NaturalProducts,selling_price_PackagedMeat,selling_price_Pharmaceutical,selling_price_PreparedFood,selling_price_Restauarant,selling_price_Salads,selling_price_Seafood,selling_price_SkinHairCare,selling_price_Travel,selling_price_Vegetablescut
0,1,2012-02,NaN,2.0,NaN,NaN,NaN,NaN,22.0,NaN,...,NaN,462.34,106.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2012-03,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,...,NaN,89.05,765.11,124.31,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2012-04,NaN,5.0,NaN,NaN,NaN,NaN,46.0,1.0,...,NaN,174.18,769.40,102.94,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2012-05,NaN,2.0,NaN,NaN,NaN,NaN,72.0,1.0,...,152.81,401.79,1634.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2012-06,NaN,5.0,NaN,NaN,NaN,NaN,78.0,NaN,...,195.19,462.34,658.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
data_index = index_df[index_cols].copy()
feature_index_df = index_df.set_index(['customer_id', 'txn_month'])
for i in range(1,4):
    print('compute previous month', i)
    feature = get_feature(data_index[index_cols], agg_txn, p=i)
    feature.set_index(['customer_id', 'txn_month'], inplace=True)
    feature_index_df = pd.concat([feature_index_df, feature], join='inner', axis=1)

compute previous month 1
delete column txn_month_y
compute previous month 2
delete column txn_month_y
compute previous month 3
delete column txn_month_y


In [171]:
txn_feature_df = feature_index_df.reset_index()

In [172]:
join_df = join_df.merge(txn_feature_df, on=['customer_id', 'txn_month'], how='left')

In [173]:
join_df.head()

,customer_id,txn_month,age_range,marital_status,rented,family_size,no_of_children,income_bracket,quantity_Alcohol_P1M,quantity_Bakery_P1M,...,selling_price_NaturalProducts_P3M,selling_price_PackagedMeat_P3M,selling_price_Pharmaceutical_P3M,selling_price_PreparedFood_P3M,selling_price_Restauarant_P3M,selling_price_Salads_P3M,selling_price_Seafood_P3M,selling_price_SkinHairCare_P3M,selling_price_Travel_P3M,selling_price_Vegetablescut_P3M
0,1,2012-04,70+,Married,0.0,2,NaN,4.0,NaN,NaN,...,NaN,89.05,765.11,124.31,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2012-05,70+,Married,0.0,2,NaN,4.0,NaN,5.0,...,NaN,174.18,769.40,102.94,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2012-06,70+,Married,0.0,2,NaN,4.0,NaN,2.0,...,152.81,401.79,1634.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2012-07,70+,Married,0.0,2,NaN,4.0,NaN,5.0,...,195.19,462.34,658.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2012-08,70+,Married,0.0,2,NaN,4.0,NaN,3.0,...,NaN,760.13,454.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transaction Frequency Feature

In [178]:
print(list(join_df.columns))

['customer_id', 'txn_month', 'age_range', 'marital_status', 'rented', 'family_size', 'no_of_children', 'income_bracket', 'quantity_Alcohol_P1M', 'quantity_Bakery_P1M', 'quantity_DairyJuicesSnacks_P1M', 'quantity_FlowersPlants_P1M', 'quantity_Fuel_P1M', 'quantity_Garden_P1M', 'quantity_Grocery_P1M', 'quantity_Meat_P1M', 'quantity_Miscellaneous_P1M', 'quantity_NaturalProducts_P1M', 'quantity_PackagedMeat_P1M', 'quantity_Pharmaceutical_P1M', 'quantity_PreparedFood_P1M', 'quantity_Restauarant_P1M', 'quantity_Salads_P1M', 'quantity_Seafood_P1M', 'quantity_SkinHairCare_P1M', 'quantity_Travel_P1M', 'quantity_Vegetablescut_P1M', 'selling_price_Alcohol_P1M', 'selling_price_Bakery_P1M', 'selling_price_DairyJuicesSnacks_P1M', 'selling_price_FlowersPlants_P1M', 'selling_price_Fuel_P1M', 'selling_price_Garden_P1M', 'selling_price_Grocery_P1M', 'selling_price_Meat_P1M', 'selling_price_Miscellaneous_P1M', 'selling_price_NaturalProducts_P1M', 'selling_price_PackagedMeat_P1M', 'selling_price_Pharmaceut